# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for a Tech Innovation Conference. I should consider factors such as size, location, amenities, and cost.

Action: Search the internet
Action Input: {"search_query": "venues for tech innovation conference in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections.
---
Title: Top 16 Must-Visit San Francisco Tech Events in 2024
Link: https://startupvoyager.com/tech-events-san-francisco/
Snippet: 11. World Agri-Tech Innovation Summit · When: 19th to 20th March 2

I need to gather all the necessary information about the San Francisco Marriott Marquis for the Tech Innovation Conference to evaluate it properly.

Action: Read website content
Action Input: {"website_url": "https://www.marriott.com/hotels/travel/sfodt-san-francisco-marriott-marquis/"}
 

Hotel near Union Square, San Francisco | San Francisco Marriott Marquis
Skip to
							main content
Menu text
Menu close text
Find & Reserve
Special Offers
Meetings & Events
Vacations
Our Brands
Our Credit Cards
About Marriott Bonvoy
Careers at Marriott
 Language Selector
English(US)
Help
Select a Language
English(US)
Deutsch
Español
Français
日本語
Close
San Francisco Marriott Marquis
Overview 
Gallery 
Accommodations 
Dining 
Experiences 
Meetings and Weddings 
San Francisco Marriott Marquis
VIEW MAP
VIEW MAP
+1 415-896-1600 
RESERVE
Cancel
DATES
 (1
 NIGHT)
 NIGHTS)
 Flexible in
Specific Dates
Flexible Dates
S
M
T
W
T
F
S
Number of Nights
-
1
+
Done
Continue with NIGHT
 NIGHTS
Rooms & Guests
1
Room, 

Final Answer: 
The San Francisco Marriott Marquis is a modern hotel located in the SoMa District, providing easy access to attractions like Moscone Center, Chase Center, Oracle Park, and Union Square shopping. The hotel offers contemporary rooms and suites with city views, popular restaurants and bars, event spaces totaling over 133,000 square feet, a fitness center, and an M Club Lounge. The hotel is pet-friendly, with a non-refundable fee of $125 per stay for pets up to 40 pounds. Valet parking is available for guests at a rate of $91.2 per day. The hotel has implemented sustainability initiatives, including the elimination of single-use plastic water bottles. Additionally, the San Francisco Marriott Marquis offers dining options at B55 Craft House & Kitchen, The View Lounge, Mission Street Pantry, and the M Club Lounge.

> Finished chain.
 

The San Francisco Marriott Marquis is a modern hotel located in the SoMa District, providing easy access to attractions like Moscone Center, Ch

 


Search results: Title: Overview | Innovation Summit San Francisco 2022
Link: https://www.medtechstrategist.com/san-francisco-22
Snippet: Innovation Summit San Francisco 2022 is MedTech's premier networking and investment conference. The Innovation Summit conference series are organized as ...
---
Title: Tech Summit | The Home of Technology
Link: https://techsummit.tech/
Snippet: Embark on a global tech exploration with Tech Summit! Our events span the globe, offering you unparalleled access to cutting-edge insights, networking ...
---
Title: San Francisco Tech Conferences - TECHSPO Silicon Valley
Link: https://techsposv.com/san-francisco-tech-conferences/
Snippet: TECHSPO Silicon Valley is a two-day technology expo held annually in San Francisco, CA, USA which brings together some of the best developers, brands, marketers ...
---
Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechusa.com/
Snippet: The annual meeting place for the global agtech ec

Exception in thread Thread-6 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: I need to effectively market the Tech Innovation Conference and engage at least 500 potential attendees. I should focus on creating compelling messages and maximizing event exposure.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference San Francisco 2022"} 


Search results: Title: Overview | Innovation Summit San Francisco 2022
Link: https://www.medtechstrategist.com/san-francisco-22
Snippet: Innovation Summit San Francisco 2022 is MedTech's premier networking and investment conference. The Innovation Summit conference series are organized as ...
---
Title: San Francisco Tech Conferences - TECHSPO Silicon Valley
Link: https://techsposv.com/san-francisco-tech-conferences/
Snippet: TECHSPO Silicon Valley is a two-day technology expo held annually in San Francisco, CA, USA which brings together some of the best developers, brands, marketers ...
---
Title: Tech Summit | The Home of Technology
Link: https://techsummit.tech/
Snippet: Embark on a g

Thought: I need to effectively market the Tech Innovation Conference and engage at least 500 potential attendees. I should focus on creating compelling messages and maximizing event exposure.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference San Francisco"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to effectively market the Tech Innovation Conference and engage at least 500 potential attendees. I should focus on creating compelling messages and maximizing event exposure.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference San Francisco"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to effectively market the Tech Innovation Conference and engage at least 500 potential attendees. I should focus on creating compelling messages and maximizing event exposure.

A

Thought: I need to effectively market the Tech Innovation Conference and engage at least 500 potential attendees. I should focus on creating compelling messages and maximizing event exposure.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference San Francisco 2022"} 


Search results: Title: Overview | Innovation Summit San Francisco 2022
Link: https://www.medtechstrategist.com/san-francisco-22
Snippet: Innovation Summit San Francisco 2022 is MedTech's premier networking and investment conference. The Innovation Summit conference series are organized as ...
---
Title: San Francisco Tech Conferences - TECHSPO Silicon Valley
Link: https://techsposv.com/san-francisco-tech-conferences/
Snippet: TECHSPO Silicon Valley is a two-day technology expo held annually in San Francisco, CA, USA which brings together some of the best developers, brands, marketers ...
---
Title: Tech Summit | The Home of Technology
Link: https://techsummit.tech/
Snippet: Embark on a g

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '780 Mission Street, San Francisco, CA 94103',
 'booking_status': '',
 'capacity': 0,
 'name': 'San Francisco Marriott Marquis'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md